## Function Calling

- "Function calling" refers to the capability of invoking specific functions or methods directly within the flow of interaction with a language model. 
- This feature allows for the execution of predefined functions or scripts in response to certain inputs or conditions identified by the language model during its processing. Essentially, it enables the language model to not only generate text-based responses but also to perform specific computational tasks, interact with external APIs, databases, or other software components, and then incorporate the results of these interactions into its output.

In [1]:
import os
import openai


In [2]:
openai_api_key = ""
open_ai_api_base = ""

openai.api_base = open_ai_api_base
openai.api_key = openai_api_key

In [3]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [4]:
# define a function
functions = [
    # Here we are passing only one function
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Cincinnati, OH",
                },
                "unit": {"type": "string", "enum": ["celsius"]},
            },
            "required": ["location"], # Required parameter
        },
    }
]

In [5]:
# Create a list of messages to pass to the language model

messages = [
    {
        "role": "user",
        "content": "What's the weather like in Cincinnati?" # Something related to the function
    }
]

In [6]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions 
    # function_call = "auto" : Auto determines that it automatically decides if it is going to use the function or not. Otherwise, we can use :
    # none  : forces to not use the function.
    # function_call = {"name": "$NAME OF THE FUNCTION"}
    # New feature added by OpenAI that allows you to define custom functions (list of functions)
)
# OBS: If you force it to call a function and the content message has nothing to do with it, it will get confused.

In [7]:
response_message = response["choices"][0]["message"]

In [8]:
response_message

<OpenAIObject at 0x10907cc70> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Cincinnati, OH\"\n}"
  }
}

In [9]:
args = json.loads(response_message["function_call"]["arguments"])

In [10]:
get_current_weather(args)

'{"location": {"location": "Cincinnati, OH"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

## OBS: 
- OpenAI models have token limits on them
- You have to not only to be aware of the length of your messages, but also to the length of your functions
- The size of the function—both in terms of its definition and its output—can significantly impact the token budget of a request. Here's why:
    1) If the function definition needs to be included with each API request (for context or instruction to the model), it consumes part of the token limit. This is more relevant during the setup or if the function's logic is dynamically adjusted based on conversation flow.
    2) The output of the function, when integrated into the conversation, also counts towards the token limit. If a function returns a large amount of text, it can quickly consume the available token budget, reducing the space for further interaction within the same API call


In [24]:
# OBS: OpenAI models have token limits on them
# You have to not only to be aware of the length of your messages, but also to the length of your functions

In [25]:
args = json.loads(response["choices"][0]["message"]['function_call']['arguments'])
args

{'location': 'Cincinnati, OH'}

In [28]:
observation = get_current_weather(args)
observation

'{"location": {"location": "Cincinnati, OH"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [29]:
messages

[{'role': 'user', 'content': "What's the weather like in Cincinnati?"}]

In [ ]:
messages.append({
    "role":"function",
    "name":"get_current_weather",
    "content":observation,
    
})